In [1]:
import sys
sys.path.append("../code/")
from wiki_dataset import get_wiki_dataset
from __future__ import division
from __future__ import print_function
import numpy as np

import chainer
import chainer.functions as F
import chainer.links as L
import os
from chainer import training
from chainer.training import extensions

# Idea
In this notebook I will train the language models that I can use in a later stadium to try to learn the linear mapping from model a to model b. In this model we get as input a word embedding and as output the next word in the sequence. 

I will train a model for spanish and english.

## Model
Our model is based on the https://github.com/pfnet/chainer/tree/master/examples/ptb model.  

In [ ]:
# This a simple language model, directly copied from the tutorial
class RNNLM(chainer.Chain):

    def __init__(self, n_vocab, n_units, train=True):
        super(RNNLM, self).__init__(
            embed=L.EmbedID(n_vocab, n_units),
            l1=L.LSTM(n_units, n_units),
            l2=L.LSTM(n_units, n_units),
            l3=L.Linear(n_units, n_vocab),
        )
        self.n_units = n_units
        self.n_vocab = n_vocab
        # Initialize with uniform distribution, expect for our linear tranformation layer
        #for param in self.params():
        #    param.data[...] = np.random.uniform(-0.1, 0.1, param.data.shape)
        
        self.train = train

    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def __call__(self, x):
        h0 = self.embed(x)
        h2 = self.l1(F.dropout(h0, train=self.train))
        h3 = self.l2(F.dropout(h2, train=self.train))
        y = self.l3(F.dropout(h3, train=self.train))
        return y

## Dataset iterator

From the tutorial we re-use the following code:
- **ParallelSequentialIteratior**: This class takes a dataset and creates batches(with batch size N) from it. In the first \_\_next()\_\_ step it will give the first words of N sequences as input and as target value the next word in the sequence. In the next \_\_next\_\_() call it will return the 2nd word of N sequences as input and the third word for as target etc. Running multiple sequences at the same time allows for efficient matrix multiplication(especially useful for GPU acceleration) furthermore the gradients for all sequences can be summed and applied immediately(allowing for statistic gradient descent). 
- **BPTTUpdater**: To obtain the weight gradients for our neural network we make use of back propagation trough time algorithm. It unfolds a neural network into a normal feedforward network where each layer represents one timestep, and then sums the gradients for each connection. The gradients are only saved for a fixed number of steps ("truncated"). 
- **Compute perplexity**: Perplexity is a measurement often used to evaluate language models in NLP context. It  assumes to obtain the softmax classification error as is used by default by the [chainer L.Classifier link](http://docs.chainer.org/en/stable/_modules/chainer/links/model/classifier.html?highlight=Classifier)

In [ ]:
# Dataset iterator to create a batch of sequences at different positions.
# This iterator returns a pair of current words and the next words. Each
# example is a part of sequences starting from the different offsets
# equally spaced within the whole sequence.
class ParallelSequentialIterator(chainer.dataset.Iterator):

    def __init__(self, dataset, batch_size, repeat=True):
        self.dataset = dataset
        self.batch_size = batch_size  # batch size
        # Number of completed sweeps over the dataset. In this case, it is
        # incremented if every word is visited at least once after the last
        # increment.
        self.epoch = 0
        # True if the epoch is incremented at the last iteration.
        self.is_new_epoch = False
        self.repeat = repeat
        length = len(dataset)
        # Offsets maintain the position of each sequence in the mini-batch.
        self.offsets = [i * length // batch_size for i in range(batch_size)]
        # NOTE: this is not a count of parameter updates. It is just a count of
        # calls of ``__next__``.
        self.iteration = 0

    def __next__(self):
        # This iterator returns a list representing a mini-batch. Each item
        # indicates a different position in the original sequence. Each item is
        # represented by a pair of two word IDs. The first word is at the
        # "current" position, while the second word at the next position.
        # At each iteration, the iteration count is incremented, which pushes
        # forward the "current" position.
        length = len(self.dataset)
        if not self.repeat and self.iteration * self.batch_size >= length:
            # If not self.repeat, this iterator stops at the end of the first
            # epoch (i.e., when all words are visited once).
            raise StopIteration
        cur_words = self.get_words()
        self.iteration += 1
        next_words = self.get_words()

        epoch = self.iteration * self.batch_size // length
        self.is_new_epoch = self.epoch < epoch
        if self.is_new_epoch:
            self.epoch = epoch

        return list(zip(cur_words, next_words))

    @property
    def epoch_detail(self):
        # Floating point version of epoch.
        return self.iteration * self.batch_size / len(self.dataset)

    def get_words(self):
        # It returns a list of current words.
        return [self.dataset[(offset + self.iteration) % len(self.dataset)]
                for offset in self.offsets]

    def serialize(self, serializer):
        # It is important to serialize the state to be recovered on resume.
        self.iteration = serializer('iteration', self.iteration)
        self.epoch = serializer('epoch', self.epoch)


# Custom updater for truncated BackProp Through Time (BPTT)
class BPTTUpdater(training.StandardUpdater):

    def __init__(self, train_iter, optimizer, bprop_len, device):
        super(BPTTUpdater, self).__init__(
            train_iter, optimizer, device=device)
        self.bprop_len = bprop_len

    # The core part of the update routine can be customized by overriding.
    def update_core(self):
        loss = 0
        # When we pass one iterator and optimizer to StandardUpdater.__init__,
        # they are automatically named 'main'.
        train_iter = self.get_iterator('main')
        optimizer = self.get_optimizer('main')

        # Progress the dataset iterator for bprop_len words at each iteration.
        for i in range(self.bprop_len):
            # Get the next batch (a list of tuples of two word IDs)
            batch = train_iter.__next__()

            # Concatenate the word IDs to matrices and send them to the device
            # self.converter does this job
            # (it is chainer.dataset.concat_examples by default)
            x, t = self.converter(batch, self.device)

            # Compute the loss at this time step and accumulate it
            loss += optimizer.target(chainer.Variable(x), chainer.Variable(t))

        optimizer.target.cleargrads()  # Clear the parameter gradients
        loss.backward()  # Backprop
        loss.unchain_backward()  # Truncate the graph
        optimizer.update()  # Update the parameters


# Routine to rewrite the result dictionary of LogReport to add perplexity
# values
def compute_perplexity(result):
    result['perplexity'] = np.exp(result['main/loss'])
    if 'validation/main/loss' in result:
        result['val_perplexity'] = np.exp(result['validation/main/loss'])

## Dataset:
First we have to divide our training, test and validation set. I use rougly the same fractions that are used in the get_ptb_words() dataset. First a 10% - 90% training - validation split. Then split the training again 10% - 90% to obtain a test and training set.

In [ ]:
def retrieve_and_split(dump):
    seq, voc = get_wiki_dataset(dump)
    seq = seq.astype(np.int32)
    
    val_start = int(len(seq) * .9)
    test_start = int(val_start *.9)
    train = seq[:test_start]
    test = seq[test_start:val_start]
    val = seq[val_start:]
    
    return train, val, test, voc

## Model training

In [ ]:
def train(dump, name, test_mode=False, epoch=5, batch_size=128, gpu=-1, out='result', grad_clip=True, brpoplen=35, resume='',max_seq_size=250000):
    """

    """
    train, val, test, voc = retrieve_and_split(dump,max_seq_size)
    n_vocab = len(voc)
    # n_vocab=10
    print("Going to run %s" % name)
    print("#training: %d, #val: %d, #test: %d" % (len(train), len(val), len(test)))
    print("#vocabulary: %d" % n_vocab)

    if test_mode:
        print("Running in test mode: cutting test, train and val set to 100 elements each")
        train = train[:10000]
        test = test[:100]
        val = val[:100]
        if batch_size > 100:
            batch_size = 100

    train_iter = ParallelSequentialIterator(train, batch_size)
    val_iter = ParallelSequentialIterator(val, 1, repeat=False)
    test_iter = ParallelSequentialIterator(test, 1, repeat=False)

    # Prepare an RNNLM model
    print("Creating model")
    rnn = RNNLM(n_vocab, 800)
    print("Init model complete")
    model = L.Classifier(rnn)
    model.compute_accuracy = False  # we only want the perplexity
    if gpu >= 0:
        chainer.cuda.get_device(gpu).use()  # make the GPU current
        model.to_gpu()

    # Set up an optimizer
    optimizer = chainer.optimizers.SGD(lr=1.0)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.GradientClipping(grad_clip))

    # Set up a trainer
    updater = BPTTUpdater(train_iter, optimizer, brpoplen, gpu)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out=out)

    eval_model = model.copy()  # Model with shared params and distinct states
    eval_rnn = eval_model.predictor
    eval_rnn.train = False
    trainer.extend(extensions.Evaluator(
        val_iter, eval_model, device=gpu,
        # Reset the RNN state at the beginning of each evaluation
        eval_hook=lambda _: eval_rnn.reset_state()))

    interval = 10 if test_mode else 500

    trainer.extend(extensions.ExponentialShift('lr', 0.5),
                   trigger=(25, 'epoch'))
    trainer.extend(extensions.LogReport(postprocess=compute_perplexity,
                                        trigger=(interval, 'iteration')))
    trainer.extend(extensions.PrintReport(
        ['epoch', 'iteration', 'perplexity', 'val_perplexity']
    ), trigger=(interval, 'iteration'))
    trainer.extend(extensions.ProgressBar(
        update_interval=10 if test_mode else 125))
    trainer.extend(extensions.snapshot())
    trainer.extend(extensions.snapshot_object(
        model, 'model_iter_{.updater.iteration}'))
    if resume:
        chainer.serializers.load_npz(resume, trainer)
    date = time.strftime("%Y-%m-%d_%H-%M-%S")
    fn_a = 'loss_%s.png' % date
    loss_r = extensions.PlotReport(['validation/main/loss','main/loss'],'epoch',file_name=fn_a)

    trainer.extend(loss_r)
    start = time.time()
    trainer.run()

    diff = time.time() - start
    
    # Evaluate the final model
    print('test')
    eval_rnn.reset_state()
    evaluator = extensions.Evaluator(test_iter, eval_model, device=gpu)
    result = evaluator()
    print('test perplexity:', np.exp(float(result['main/loss'])))
    com.add_text("final loss",result['main/loss'])



I actually executed this script via a batch script for english and spanish. Using the `sh scripts/english_lm.sh` and `sh scripts/spanish_lm.sh`. Next I will read the results.

## Analyse learned language models

In [47]:
from models import load_rnn_model
import re
from chainer.functions.activation.softmax import softmax
def tokenize(line):
    line = line.replace("<br>", " ").replace(". ", " <eos> ").lower()
    for token in re.findall("[\w\<\>]+", line):
        yield token
def read_dataset(file):
    """
    Reads dataset that was exported by export_dataset
    :param file:
    :return:
    """
    np.load(file)
    data = np.load(file)
    seq,voc = data['seq'],data['voc']
    seq = seq.astype(np.int32)
    return seq,voc

#### Load datasets and models

In [48]:
ENGLISH_FOLDER = "../result-english/"
SPANISH_FOLDER = "../result-spanish/"
model_english = os.path.join(ENGLISH_FOLDER,'model_iter_5430')
model_spanish = os.path.join(SPANISH_FOLDER,'model_iter_5432')
seq_en, voc_en = read_dataset(os.path.join(ENGLISH_FOLDER,'data.npz'))
seq_es, voc_es = read_dataset(os.path.join(SPANISH_FOLDER,'data.npz'))
rnn_en = load_rnn_model(model_english,len(voc_en),800)
rnn_es = load_rnn_model(model_spanish,len(voc_es),800)
print("The english vocabulary contains %d different words" % len(voc_en))
print("The spanish vocabulary contains %d different words" % len(voc_es))

The english vocabulary contains 8981 different words
The spanish vocabulary contains 8186 different words


In [68]:

def create_inverse_voc(voc):
    return dict([(word,idx) for idx,word in enumerate(voc)])

def map_line_to_seq(line,inverse_voc):
    return [inverse_voc[w] if w in inverse_voc else inverse_voc['<below_th>'] for w in tokenize(line)]
def map_seq_to_sentence(seq, voc):
    return " ".join([voc[int(w)] for w in seq])

def generate_text(pred, voc, seeds=[],max_len=100):
    inverse_voc = create_inverse_voc(voc)
    text_idx = [map_line_to_seq(s,inverse_voc) for s in seeds]
    n= len(text_idx)

    for i in range(max_len):
        X = []
        for s in text_idx:
            X.append(s[i])
        X = np.array(X,dtype=np.int32)
        d = pred(X).data
        d = np.delete(d,0,1)
        next_words = np.argmax(d,axis=1) + 1
        next_index = i + 1
        for si in range(len(text_idx)):
            if len(text_idx[si]) <= next_index:
                text_idx[si].append(next_words[si])
    
    return [map_seq_to_sentence(s, voc) for s in text_idx]
seeds = ["Sheep eat a lot of grass and produce wool. ", "The Golden Rule or law of reciprocity is the principle of treating others as one would wish"]
print(voc_en[:15])
rnn_en.predictor.train = False
generate_text(rnn_en.predictor, voc_en, seeds)

['<below_th>' 'Kutch' 'Gurjar' 'contributions' 'to' 'the' 'Indian'
 'railways' 'The' 'KGK' 'were' 'widespread' 'from' 'late' 'latest']


['sheep <below_th> a lot of grass and produce <below_th> <eos> The film was also known as the Hayseeds s director of the Year and the National Film Festival of the Year in the United States <eos> The album was released on August 2011 <eos> It was released on August 2011 <eos> It was released on August 29 2011 <eos> It was released on August 29 September 2011 <eos> The following year the season was released on August 2011 <eos> The album was released on August 2011 <eos> The album was released on August 29 2011 <eos> The following year the season',
 'the <below_th> rule or law of <below_th> is the principle of <below_th> others as one would <below_th> the author s own and the film is not known as a result of the film s own work and the film was released on the film as a result of the film s first film was released on August 2011 <eos> The film was released on August 2011 <eos> The album was released on August 29 2011 <eos> The album was released on August 29 2011 <eos> The following yea